In [ ]:
import gpfanova, scipy
import GPy
import matplotlib as mpl
from GPy.kern import RBF
from derivative import RBFDerivative
import pandas as pd

In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
melt = pd.read_csv("data/normalized/lund/propionicAcid-ecoli/tidy.csv")

In [ ]:
# melt = melt[melt.propionicAcidmM>0]

xgp = melt[['time','pH','propionicAcidmM']]
ygp = melt.od

# xgp.propionicAcidmM = np.log10(xgp.propionicAcidmM)

xgp = xgp.values
ygp = ygp.values[:,None]

step = 3

xgp = xgp[::step,:]
ygp = ygp[::step,:]

# select = np.isnan(ygp)

# ygp = ygp[~select][:,None]
# xgp = xgp[~select,:]

plt.figure(figsize=(20,8))

plt.subplot(211)
plt.plot(xgp)

plt.subplot(212)
plt.plot(ygp)

In [ ]:
xgp.shape, ygp.shape

In [ ]:
#kern = RBF(1, name='mean') + RBF(2,active_dims=[0,1],name='pH',ARD=True) + RBF(2,active_dims=[0,2],name='PA',ARD=True) + RBF(3,name='interaction',ARD=True)
kern = RBF(3, ARD=True)

kern

In [ ]:
gp = GPy.models.GPRegression(xgp,ygp,kern)
gp

In [ ]:
gp.optimize()
gp

In [ ]:
xpred = np.zeros((50,3))
xpred[:,0] = np.linspace(melt.time.min(),melt.time.max())

In [ ]:

plt.figure(figsize=(np.unique(xgp[:,2]).shape[0]*5./3, 5*3))

for i,ph in enumerate(np.unique(xgp[:,1])):
    for j,pa in enumerate(np.unique(xgp[:,2])):
        
        plt.subplot(3, np.unique(xgp[:,2]).shape[0]/3+1, j + 1)
        
        plt.title("%.2lf mM PA"%((pa)))
        
        xpred[:,1:] = ph, pa

        mu,cov = gp.predict_noiseless(xpred,full_cov=True)
        mu = mu[:,0]

        std = np.sqrt(cov.diagonal())
        
        color = plt.get_cmap()(1.*(ph-4)/3)

        plt.plot(xpred[:,0],mu, label = ph,c=color)
        plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)
        
        #select = g.get_group((np.round(10**ph,2),pa)).index
        #plt.scatter(x,np.nanmean(y[:,select],1),c=color,marker='x',s=20)
        
        #plt.ylim(-1.2,4.4)
        
plt.subplot(3, np.unique(xgp[:,2]).shape[0]/3+1,1)
plt.legend(loc='best')
plt.tight_layout()
# plt.savefig("figures/lund/ecoli-gp-byPA.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(np.unique(xgp[:,1]).shape[0]*5./3, 5*3))

for i,ph in enumerate(np.unique(xgp[:,1])):
    for j,pa in enumerate(np.unique(xgp[:,2])):
        
        plt.subplot(3, np.unique(xgp[:,1]).shape[0]/3+1, i + 1)
        
        plt.title("pH=%.2lf"%(ph))
        
        xpred[:,1:] = ph, pa

        mu,cov = gp.predict_noiseless(xpred,full_cov=True)
        mu = mu[:,0]

        std = np.sqrt(cov.diagonal())
        
        color = plt.get_cmap()(pa/50)

        plt.plot(xpred[:,0],mu, label = np.round(pa,2),c=color)
        plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)
        
        #select = g.get_group((np.round(10**ph,2),pa)).index
        #plt.scatter(x,np.nanmean(y[:,select],1),c=color,marker='x',s=20)
        
        plt.ylim(-1.2,4.4)
        
plt.subplot(3, np.unique(xgp[:,1]).shape[0]/3+1,1)
plt.legend(loc='best')
plt.tight_layout()
plt.savefig("figures/lund/ecoli-gp-byPA.pdf",bbox_inches='tight')

## Outlier detection 

In [ ]:
g = melt.groupby(['pH', 'propionicAcidmM','rep'])
len(g.groups.keys())

In [ ]:
ll = []

for k,temp in g:
    
    temp.sort_values('time',inplace=True)
    
    xpred = temp[['time','pH','propionicAcidmM']].values
    mu,cov = gp.predict(xpred,full_cov=True)

    l = scipy.stats.multivariate_normal.logpdf(temp.od, mu[:,0], cov)
    
    diff = temp.od-mu[:,0]
    ts = np.dot(diff, np.dot(np.linalg.inv(cov), diff))
    p = 1-scipy.stats.chi2.cdf(ts, df=mu.shape[0])
    
    ll.append((k, l, ts, p))

In [ ]:
ll = pd.DataFrame(ll, columns=['design','ll', 'testStatistic','pval'])
ll.head()

In [ ]:
ll.ll.hist(bins=20)
plt.savefig("figures/lund/ecoli-outliers-loglikelihood.pdf")

In [ ]:
ll.pval.hist(bins=20)
plt.savefig("figures/lund/ecoli-outliers-pvals.pdf")

In [ ]:
phVals = np.unique(xgp[:,1]).tolist()
paVals = np.unique(xgp[:,2]).tolist()

cmap = plt.get_cmap()

fig = plt.figure(figsize=(5*len(paVals), 5*len(phVals)))

for i, r in ll.iterrows():
    design, l = r.design, r.ll
    ph, pa, rep = design
    
    plt.subplot(len(phVals), len(paVals), phVals.index(ph)*len(paVals) + paVals.index(pa) + 1)
    plt.title((ph,pa),fontsize=20)
    
    temp = g.get_group(design)
    temp.sort_values('time',inplace=True)
    plt.plot(temp.time,temp.od,color=cmap((l-ll.ll.min())/(ll.ll.max()-ll.ll.min())))
    
fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

norm = mpl.colors.Normalize(vmin=ll.ll.min(), vmax=ll.ll.max())
cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb.set_label('Log-likelihood',fontsize=20)
[t.set_fontsize(20) for t in cb.ax.get_yticklabels()]

plt.savefig('figures/lund/ecoli-outliers-byLL.pdf',bbox_inches='tight')

In [ ]:
phVals = np.unique(xgp[:,1]).tolist()
paVals = np.unique(xgp[:,2]).tolist()

cmap = plt.get_cmap()

for thresh in [-5000, -1000, -500]:

    fig = plt.figure(figsize=(5*len(paVals), 5*len(phVals)))

    for i, r in ll.iterrows():
        design, l = r.design, r.ll
        ph, pa, rep = design

        plt.subplot(len(phVals), len(paVals), phVals.index(ph)*len(paVals) + paVals.index(pa) + 1)
        plt.title((ph,pa),fontsize=20)
        
        c='r'
        if l > thresh:
            c = 'k'

        temp = g.get_group(design)
        temp.sort_values('time',inplace=True)
        plt.plot(temp.time,temp.od,color=c)

    plt.savefig('figures/lund/ecoli-outliers-byLL-thresh%d.pdf'%thresh,bbox_inches='tight')

In [ ]:
phVals = np.unique(xgp[:,1]).tolist()
paVals = np.unique(xgp[:,2]).tolist()

cmap = plt.get_cmap()

fig = plt.figure(figsize=(5*len(paVals), 5*len(phVals)))

for i, r in ll.iterrows():
    design, l = r.design, r.pval
    ph, pa, rep = design
    
    plt.subplot(len(phVals), len(paVals), phVals.index(ph)*len(paVals) + paVals.index(pa) + 1)
    plt.title((ph,pa),fontsize=20)
    
    temp = g.get_group(design)
    temp.sort_values('time',inplace=True)
    plt.plot(temp.time,temp.od,color=cmap(l))
    
fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

norm = mpl.colors.Normalize(vmin=0, vmax=1)
cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb.set_label('Log-likelihood',fontsize=20)
[t.set_fontsize(20) for t in cb.ax.get_yticklabels()]

plt.savefig('figures/lund/ecoli-outliers-byPval.pdf',bbox_inches='tight')

In [ ]:
phVals = np.unique(xgp[:,1]).tolist()
paVals = np.unique(xgp[:,2]).tolist()

cmap = plt.get_cmap()

fig = plt.figure(figsize=(5*len(paVals), 5*len(phVals)))

for i, r in ll.iterrows():
    design, l = r.design, -np.log10(r.pval+1e-15)
    ph, pa, rep = design
    
    plt.subplot(len(phVals), len(paVals), phVals.index(ph)*len(paVals) + paVals.index(pa) + 1)
    plt.title((ph,pa),fontsize=20)
    
    temp = g.get_group(design)
    temp.sort_values('time',inplace=True)
    plt.plot(temp.time,temp.od,color=cmap(l))
    
fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])

norm = mpl.colors.Normalize(vmin=0, vmax=15)
cb = mpl.colorbar.ColorbarBase(cbar_ax, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb.set_label('Log-likelihood',fontsize=20)
[t.set_fontsize(20) for t in cb.ax.get_yticklabels()]

plt.savefig('figures/lund/ecoli-outliers-byPvalLog10.pdf',bbox_inches='tight')

In [ ]:
l

In [ ]:
for ph in np.unique(xgp[:,1]):

    xpred[:,1] = ph

    mu,var = gp.predict_noiseless(xpred, kern=kern.pH)
    mu = mu[:,0]
    var = var[:,0]
    std = np.sqrt(var)

    plt.plot(xpred[:,0],mu, label = np.round(10**ph,1),c=color)
    #plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)

In [ ]:
for pa in np.unique(xgp[:,2]):

    xpred[:,2] = pa

    mu,var = gp.predict_noiseless(xpred, kern=kern.PA)
    mu = mu[:,0]
    var = var[:,0]
    std = np.sqrt(var)

    plt.plot(xpred[:,0],mu, label = np.round(10**ph,1),c=color)
    #plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)

In [ ]:
# plt.figure(figsize=(12*4,7*4))

# for i,ph in enumerate(np.unique(xgp[:,1])):
#     for j,pa in enumerate(np.unique(xgp[:,2])):
        
#         plt.subplot(7,12,i*12+j+1)
#         plt.title("pH %.2lf, %.2lf mM PA"%(10**ph,10**pa-1e-1))

#         xpred[:,1] = ph
#         xpred[:,2] = pa

#         mu,var = gp.predict_noiseless(xpred, kern=kern.interaction)
#         mu = mu[:,0]
#         var = var[:,0]
#         std = np.sqrt(var)

#         plt.plot(xpred[:,0],mu, label = np.round(10**ph,1))
#         plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1)
        
#         plt.plot([x.min(),x.max()],[0,0],c='k',lw=3)
#         plt.ylim(-2.6,2.6)
        
# plt.tight_layout()

In [ ]:
# plt.figure(figsize=(12*4,7*4))

# basePh = np.unique(xgp[:,1]).max()
# basePa = np.unique(xgp[:,2]).min()

# phVals = np.unique(xgp[:,1])
# phVals = phVals[phVals!=basePh]

# paVals = np.unique(xgp[:,2])
# paVals = paVals[paVals!=basePa]

# xpred = np.zeros((200,3))
# xpred[:,0] = np.tile(np.linspace(x.min(),x.max()),4)

# op = np.zeros((50,200))
# for i in range(50):
#     #op[i,i] = 1
#     #op[i,50+i::50] = -1
#     op[i,i::50] = [1,-1,-1,1]

# for i,ph in enumerate(phVals):
#     for j,pa in enumerate(paVals):
        
#         plt.subplot(7,12,i*12+j+1)
#         plt.title("pH %.2lf, %.2lf mM PA"%(10**ph,10**pa-1e-1))
        
#         xpred[:,1] = np.repeat([ph,ph,basePh,basePh], 50)
#         xpred[:,2] = np.repeat([pa,basePa,pa,basePa], 50)

#         mu,cov = gp.predict_noiseless(xpred, kern=kern.interaction, full_cov=True)
        
#         mu = np.dot(op, mu)
#         cov = np.dot(op, np.dot(cov, op.T))
                
#         mu = mu[:,0]
#         std = np.sqrt(cov.diagonal())

#         plt.plot(xpred[:50,0],mu, label = np.round(10**ph,1))
#         plt.fill_between(xpred[:50,0], mu-2*std, mu+2*std,alpha=.1)
        
#         plt.plot([x.min(),x.max()],[0,0],c='k',lw=3)
#         plt.ylim(-4.8,4.8)
        
# plt.tight_layout()


In [ ]:
# mu = np.zeros(xpred.shape[0])
# var = np.zeros(xpred.shape[0])
# for k in [kern.mean, kern.pH, kern.PA, kern.interaction]:
#     mutemp,vartemp = gp.predict_noiseless(xpred, kern=k)
    
#     mu += mutemp[:,0]
#     var += vartemp[:,0]
    
# plt.plot(xpred[:,0],mu)
# std = np.sqrt(var)
# #plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)
# mu,var = gp.predict_noiseless(xpred)
# mu = mu[:,0]
# var = var[:,0]
# plt.plot(xpred[:,0],mu)
# std = np.sqrt(var)
# plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)

In [ ]:
# kern = RBF(1, name='mean') + RBF(2,active_dims=[0,1],name='pH',ARD=True) + RBF(2,active_dims=[0,2],name='PA',ARD=True) + RBF(3,name='interaction',ARD=True)
kerncopy = kern.copy()

kerncopy

In [ ]:
class RBFDerivative(RBF):
    
    def K(self,X,X2):
        k = super(RBFDerivative, self).K(X,X2)
        if X2 is None:
            diff = np.zeros((X.shape[0],X.shape[0]))
            for i in range(X.shape[0]):
                for j in range(X.shape[0]):
                    diff[i,j] = X[i,0] - X[j,0]
                    #diff[i,j] = -X[i,0] + X[j,0]
            
            return (1./(self.lengthscale[0]))*(1-(1./(self.lengthscale[0]))*(diff**2))*k
            #return k * (-1./(self.lengthscale[0]))*diff
        else:
            #X2 is derivative obs
            
            diff = np.zeros((X.shape[0],X2.shape[0]))
            for i in range(X.shape[0]):
                for j in range(X2.shape[0]):
                    diff[i,j] = X[i,0] - X2[j,0]
                    #diff[i,j] = -X[i,0] + X[j,0]
                    
            return k * (1./(self.lengthscale[0]))*diff        
            #return (1./(self.lengthscale[0]))*(1-(1./(self.lengthscale[0]))*(diff**2))*k

In [ ]:
# kernDeriv = RBFDerivative(1, variance=kern.mean.variance.copy(), lengthscale = kern.mean.lengthscale.copy()) +\
#             RBFDerivative(2, ARD=True, variance=kern.pH.variance.copy(), lengthscale = kern.pH.lengthscale.copy()) +\
#             RBFDerivative(2, ARD=True, active_dims=[0,2], variance=kern.PA.variance.copy(), lengthscale = kern.PA.lengthscale.copy()) +\
#             RBFDerivative(3, ARD=True, variance=kern.interaction.variance.copy(), lengthscale = kern.interaction.lengthscale.copy())
kernDeriv = RBFDerivative(3, ARD=True, variance=kern.variance.copy(), lengthscale = kern.lengthscale.copy())
kernDeriv

In [ ]:
kern

In [ ]:
K = kernDeriv.K(xgp[:1000,:], xgp[:1000,:])

plt.figure(figsize=(6,6))
plt.imshow(K)
plt.colorbar()

In [ ]:
K = kernDeriv.K(xgp[:100,:])

plt.figure(figsize=(6,6))
plt.imshow(K)
plt.colorbar()

In [ ]:
plt.figure(figsize=(np.unique(xgp[:,2]).shape[0]*5./3, 5*3))

for i,ph in enumerate(np.unique(xgp[:,1])):
    for j,pa in enumerate(np.unique(xgp[:,2])):
        
        plt.subplot(3, np.unique(xgp[:,2]).shape[0]/3+1, j + 1)
        
        plt.title("%.2lf mM PA"%((pa)))
        
        xpred[:,1:] = ph, pa

        mu,cov = gp.predict_noiseless(xpred,full_cov=False,kern=kernDeriv)
        mu = mu[:,0]

        cov = cov[:,0]
        std = np.sqrt(cov)
        
        color = plt.get_cmap()(1.*(ph-4)/3)

        plt.plot(xpred[:,0],mu, label = ph,c=color)
        plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)
        
        #select = g.get_group((np.round(10**ph,2),pa)).index
        #plt.scatter(x,np.nanmean(y[:,select],1),c=color,marker='x',s=20)
        
        plt.ylim(-3.4,6.4)
        plt.ylabel("d logOD / dt")

        
plt.subplot(3, np.unique(xgp[:,2]).shape[0]/3+1,1)
plt.legend(loc='best')
plt.tight_layout()
plt.savefig("figures/lund/ecoli-gp-byPA-derivative.pdf",bbox_inches='tight')

In [ ]:
plt.figure(figsize=(np.unique(xgp[:,1]).shape[0]*5./3, 5*3))

for i,ph in enumerate(np.unique(xgp[:,1])[::-1]):
    for j,pa in enumerate(np.unique(xgp[:,2])):
        
        plt.subplot(3, np.unique(xgp[:,1]).shape[0]/3+1, i + 1)
        
        plt.title("ph=%.2lf"%(ph))
        
        xpred[:,1:] = ph, pa

        mu,cov = gp.predict_noiseless(xpred,full_cov=False,kern=kernDeriv)
        mu = mu[:,0]
        cov = cov[:,0]
        std = np.sqrt(cov)
        
        color = plt.get_cmap()(1.*(pa)/50)

        plt.plot(xpred[:,0],mu, label = pa ,c=color)
        plt.fill_between(xpred[:,0], mu-2*std, mu+2*std,alpha=.1,color=color)
        
        plt.ylim(-3.4,6.4)
        plt.ylabel("d logOD / dt")

        
plt.subplot(3, np.unique(xgp[:,1]).shape[0]/3+1,1)
plt.legend(loc='best')
plt.tight_layout()
plt.savefig("figures/lund/ecoli-gp-byPH-derivative.pdf",bbox_inches='tight')

In [ ]:
phVals = np.unique(xgp[:,1])
paVals = np.unique(xgp[:,2])

In [ ]:
xpred = np.zeros((phVals.shape[0]*paVals.shape[0], 3))
xpred[:,1] = np.repeat(phVals, paVals.shape[0])
xpred[:,2] = np.tile(paVals, phVals.shape[0])

plt.figure(figsize=(15,5))

ntp = 10
nrow = 2
ncol = ntp/nrow

for i,xtemp in enumerate(np.linspace(melt.time.min(),melt.time.max(), ntp)):
    xpred[:,0] = xtemp

    mu,cov = gp.predict_noiseless(xpred,kern=kernDeriv)
    mu = mu[:,0]
    cov = cov[:,0]
    std = np.sqrt(cov)

    mu = mu.reshape((phVals.shape[0], paVals.shape[0]))

    plt.subplot2grid((2,1+4*ncol),(i/ncol, (i%ncol)*4),colspan=4)
    plt.imshow(mu,vmax=6.4,vmin=-6.4,cmap='PRGn')
    im = plt.title("time=%.2lf"%xtemp)
    
    plt.yticks(range(0,phVals.shape[0]), phVals[::].astype(str))
    plt.xticks(range(0,paVals.shape[0],2), (paVals[::2]).astype(str))
    
ax = plt.subplot2grid((2,1+4*ncol),(0,4*ncol),rowspan=nrow)
plt.colorbar(cax=ax)
    
plt.tight_layout()

plt.savefig("figures/lund/ecoli-gp-derivative-vsPaPh-byTime.pdf")

In [ ]:
phVals

In [ ]:
xpred = np.zeros((50,3))
xpred[:,0] = 3#melt.time.min()
xpred[:,1] = np.linspace(phVals[0], phVals[-1])

plt.figure(figsize=(10,8))
for pa in paVals:
# xpred[:,2] = np.log10(ds.meta.propionicAcidmM.unique()+1e-1)[0]
    xpred[:,2] = pa

    mu,cov = gp.predict_noiseless(xpred,kern=kernDeriv)
    mu = mu[:,0]
    cov = cov[:,0]
    std = np.sqrt(cov)

    color = plt.get_cmap()(((pa))/50)
    
    plt.plot(xpred[:,1],mu,color=color,label=pa)
    plt.fill_between(xpred[:,1],mu-2*std,mu+2*std,alpha=.1,color=color)
    
plt.scatter(phVals,[0]*phVals.shape[0],marker='x',c='k')
plt.plot([phVals.min(), phVals.max()],[0,0],c='k')

plt.legend(loc='best')
plt.tight_layout()
plt.savefig("figures/lund/ecoli-gp-derivative-vsPh-byPa.pdf")

In [ ]:
xpred = np.zeros((50,3))
xpred[:,0] = 3#melt.time.min()
xpred[:,2] = np.linspace(paVals[0], paVals[-1])

plt.figure(figsize=(10,8))
for pa in phVals:
# xpred[:,2] = np.log10(ds.meta.propionicAcidmM.unique()+1e-1)[0]
    xpred[:,1] = pa

    mu,cov = gp.predict_noiseless(xpred,kern=kernDeriv)
    mu = mu[:,0]
    cov = cov[:,0]
    std = np.sqrt(cov)

    color = plt.get_cmap()(((pa-3))/4)
    
    plt.plot(xpred[:,2],mu,color=color,label=pa)
    plt.fill_between(xpred[:,2],mu-2*std,mu+2*std,alpha=.1,color=color)
    
plt.scatter(paVals,[0]*paVals.shape[0],marker='x',c='k')
plt.plot([paVals.min(), paVals.max()],[0,0],c='k')

plt.legend(loc='best')
plt.tight_layout()
plt.savefig("figures/lund/ecoli-gp-derivative-vsPa-byPh.pdf")

In [ ]:
ds = gpfanova.dataset.DataSet('data/normalized/lund/ecoli-replicate/')

temp = ds.meta.position.str.extract('([A-Z]?)([0-9]{1,2})\.?[0-9]?')
temp.columns = ['row','column']

row = None
for i,r in temp.iterrows():
    if r.row != '':
        row = r.row
    else:
         temp.loc[i,'row'] = row
            
ds.meta = temp

x,y,effect,labels = ds.build(effects=['row','column'],scale='range')
# y = (y-y.mean())/y.std()

dm = np.ones((y.shape[1],1))

In [ ]:
effect